<a href="https://colab.research.google.com/github/mogollonalex/Econometrics-R/blob/main/Econometrics_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#     i:,:,,:i                                                              
#    ,@B@B@@@@                                                              
# iE2@G0qENEqMOFP                                           J       :,      
# NB@q        @B@,  rr7:  ir7,  vi7:  :r7i  rrr::rr   i;r. .@;  7r: i  :i:, 
# u@BB2SSPSX5kB@B. vL  i ru  G .8  E. G  1i M. X7 i5 7L ,M  N  :N   k. ::iM 
# FB@BEGOOOOOEF58  Ji    77  N .1  1: Z  77 0  L: .F LX:i.  G  :L   P ,F  Z 
# 1@B@...,.,...,:  .Jvvv  Y77v .7  v. 77ru  Y  r, .v  vrr:  ;j .;   L  jiiX.
#    U@@@B@@@@@B@:                                                          
#     kJvLvLLLvuP                                                           
                                                                               
####################################################################################################
############                   Elaborado por W. Alexander Mogollón B.           ####################
####################################################################################################

####################################################################################################
############                              Econometría con R                     ####################
####################################################################################################


In [6]:
install.packages(c("car","TSA","tseries","lmtest","fBasics","rgl",
                  "forecast","openxlsx","timsac","strucchange",
                  "sandwich","nlme","foreign","gplots","plm"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘xts’, ‘TTR’, ‘bitops’, ‘miscTools’, ‘rbibutils’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘leaps’, ‘locfit’, ‘quadprog’, ‘zoo’, ‘quantmod’, ‘timeDate’, ‘timeSeries’, ‘gss’, ‘stabledist’, ‘htmlwidgets’, ‘fracdiff’, ‘Rcpp’, ‘urca’, ‘RcppArmadillo’, ‘gtools’, ‘caTools’, ‘bdsmatrix’, ‘collapse’, ‘maxLik’, ‘Rdpack’, ‘Formula’




In [17]:
### Factor inlfador de varianza (multicolinealidad)
library(car)
###Heterocedásticidad 
library(lmtest)
### MCG
library(nlme)

Loading required package: carData

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [18]:
library(car)
library(TSA)
library(tseries)
library(lmtest)
library(fBasics)
library(rgl)
library(forecast)
library(openxlsx)
library(timsac)
library(strucchange) 
library(sandwich) 
library(nlme)
library(foreign)
library(gplots)
library(plm)


Attaching package: ‘TSA’


The following objects are masked from ‘package:stats’:

    acf, arima


The following object is masked from ‘package:utils’:

    tar


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘fBasics’


The following objects are masked from ‘package:TSA’:

    kurtosis, skewness


The following object is masked from ‘package:car’:

    densityPlot


Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”
Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”
Registered S3 methods overwritten by 'forecast':
  method       from
  fitted.Arima TSA 
  plot.Arima   TSA 


Attaching package: ‘forecast’


The following object is masked from ‘package:nlme’:

    getResponse


Loading required package: sandwich


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess




In [19]:
path = "https://raw.githubusercontent.com/mogollonalex/data/main/datosec.txt"
datos <-read.table(path,  header=TRUE, sep=";", na.strings="NA", dec=".", strip.white=TRUE)
datos

year,PIBpc,GCF,GCFG,FBK
<int>,<dbl>,<dbl>,<dbl>,<dbl>
1990,147882238078,105490320958,12144357697,18363200804
1991,150842260190,107147896658,12539020718,16758737154
1992,156943723177,110399522432,13863578764,23962467704
1993,165395786043,117133872195,14339819344,33116517191
1994,175012991811,124071672809,16523542144,41962822166
1995,184117933489,130825669369,17722649328,44508244241
1996,187903130700,132190126268,21965345540,39158380056
1997,194348759914,135371580723,25426350048,38905873357
1998,195456128227,134279180072,25948933373,36451637974


In [20]:
year <-datos$year
year
PIBpc <-datos$PIBpc
PIBpc
GCF <-datos$GCF
GCF
GCFG <- datos$GCFG
GCFG
FBK <- datos$FBK
FBK

[1] 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
[16] 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019

[1] 147882238078 150842260190 156943723177 165395786043 175012991811
 [6] 184117933489 187903130700 194348759914 195456128227 187239122802
[11] 192715607786 195949179708 200855708890 208725781698 219857173696
[16] 230473551393 245954157224 262527027189 271146960851 274237083517
[21] 286563105192 306473200201 318464380249 334814320894 349877717639
[26] 360219757906 367738922100 372737821103 382113386673 394571111125

[1] 105490320958 107147896658 110399522432 117133872195 124071672809
 [6] 130825669369 132190126268 135371580723 134279180072 127033144119
[11] 129570158734 131576148117 134207804058 138156617761 143494397927
[16] 149855145739 159399714623 169844941547 176834065124 179854879065
[21] 188942243676 199335476505 210565771291 220308433380 229669518970
[26] 236772084209 240521674884 245461543599 252845134772 264097336859

[1] 12144357697 12539020718 13863578764 14339819344 16523542144 17722649328
 [7] 21965345540 25426350048 25948933373 26892823715 26817379973 27411434017
[13] 27254920268 27751225948 29524660548 31053258105 32672219440 34164982100
[19] 35769823304 37473505429 39431791176 41985769974 44004066280 47918431504
[25] 50159119883 52592195055 53551481493 55502265921 59404276030 61931138687

[1] 18363200804 16758737154 23962467704 33116517191 41962822166 44508244241
 [7] 39158380056 38905873357 36451637974 22370379191 25113726850 27229296932
[13] 29971387194 33374669140 37107026249 41364243651 48368786064 56009628073
[19] 61097162930 57250746177 62725116846 74339253938 76485214872 82456128180
[25] 92312571109 91249845845 91101503136 88160406199 89980108569 93539625352

In [21]:
year=ts(year, start=1990, frequency=1)
year
PIBpc=ts(PIBpc, start=1990, frequency=1)
PIBpc
GCF=ts(GCF, start=1990, frequency=1)
GCF
GCFG=ts(GCFG, start=1990, frequency=1)
GCFG
FBK=ts(FBK, start=1990, frequency=1)
FBK

A Time Series:
 [1] 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
[16] 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019

A Time Series:
 [1] 147882238078 150842260190 156943723177 165395786043 175012991811
 [6] 184117933489 187903130700 194348759914 195456128227 187239122802
[11] 192715607786 195949179708 200855708890 208725781698 219857173696
[16] 230473551393 245954157224 262527027189 271146960851 274237083517
[21] 286563105192 306473200201 318464380249 334814320894 349877717639
[26] 360219757906 367738922100 372737821103 382113386673 394571111125

A Time Series:
 [1] 105490320958 107147896658 110399522432 117133872195 124071672809
 [6] 130825669369 132190126268 135371580723 134279180072 127033144119
[11] 129570158734 131576148117 134207804058 138156617761 143494397927
[16] 149855145739 159399714623 169844941547 176834065124 179854879065
[21] 188942243676 199335476505 210565771291 220308433380 229669518970
[26] 236772084209 240521674884 245461543599 252845134772 264097336859

A Time Series:
 [1] 12144357697 12539020718 13863578764 14339819344 16523542144 17722649328
 [7] 21965345540 25426350048 25948933373 26892823715 26817379973 27411434017
[13] 27254920268 27751225948 29524660548 31053258105 32672219440 34164982100
[19] 35769823304 37473505429 39431791176 41985769974 44004066280 47918431504
[25] 50159119883 52592195055 53551481493 55502265921 59404276030 61931138687

A Time Series:
 [1] 18363200804 16758737154 23962467704 33116517191 41962822166 44508244241
 [7] 39158380056 38905873357 36451637974 22370379191 25113726850 27229296932
[13] 29971387194 33374669140 37107026249 41364243651 48368786064 56009628073
[19] 61097162930 57250746177 62725116846 74339253938 76485214872 82456128180
[25] 92312571109 91249845845 91101503136 88160406199 89980108569 93539625352

In [22]:
###Modelo de regresión múltiple
mod1=lm(PIBpc~GCF+GCFG+FBK, x=T)
summary(mod1)


Call:
lm(formula = PIBpc ~ GCF + GCFG + FBK, x = T)

Residuals:
       Min         1Q     Median         3Q        Max 
-7.974e+09 -3.041e+09  2.045e+08  2.650e+09  7.312e+09 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.661e+10  1.201e+10   1.383  0.17837    
GCF         1.007e+00  2.027e-01   4.968 3.66e-05 ***
GCFG        1.451e+00  4.034e-01   3.596  0.00133 ** 
FBK         3.216e-01  2.011e-01   1.600  0.12176    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.284e+09 on 26 degrees of freedom
Multiple R-squared:  0.9973,	Adjusted R-squared:  0.997 
F-statistic:  3234 on 3 and 26 DF,  p-value: < 2.2e-16


In [24]:
#Normalidad
res=residuals(mod1)
shapiro.test(res)
AIC(mod1)
BIC(mod1)



	Shapiro-Wilk normality test

data:  res
W = 0.96895, p-value = 0.511


[1] 1421.527

[1] 1428.533

In [25]:
####Matriz de correlaciones simples
m=data.frame(PIBpc,GCF,GCFG,FBK)
m

PIBpc,GCF,GCFG,FBK
<ts>,<ts>,<ts>,<ts>
147882238078,105490320958,12144357697,18363200804
150842260190,107147896658,12539020718,16758737154
156943723177,110399522432,13863578764,23962467704
165395786043,117133872195,14339819344,33116517191
175012991811,124071672809,16523542144,41962822166
184117933489,130825669369,17722649328,44508244241
187903130700,132190126268,21965345540,39158380056
194348759914,135371580723,25426350048,38905873357
195456128227,134279180072,25948933373,36451637974


In [26]:
#Correlacion
cor(m)

,PIBpc,GCF,GCFG,FBK
PIBpc,1.0000000,0.9979141,0.9864754,0.9717354
GCF,0.9979141,1.0000000,0.9818698,0.9765443
GCFG,0.9864754,0.9818698,1.0000000,0.9303614
FBK,0.9717354,0.9765443,0.9303614,1.0000000


In [27]:
#Multicolinealidad
vif(mod1)

GCF      GCFG       FBK 
157.25474  54.23371  42.03326

In [28]:
#Pruebas para detectar heterocedásticidad 
# Prueba de Breush-Pagan-Godfrey (BPG)
bptest(mod1)


	studentized Breusch-Pagan test

data:  mod1
BP = 1.582, df = 3, p-value = 0.6635


In [29]:
#Pruebas para detectar autocorrelacionados. 
# Test de Breush Godfrey
bgtest(mod1, order = 2, order.by = NULL, type = c("Chisq"),
data = datos)


	Breusch-Godfrey test for serial correlation of order up to 2

data:  mod1
LM test = 24.86, df = 2, p-value = 3.996e-06


In [30]:
# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)


	RESET test

data:  mod1
RESET = 15.953, df1 = 2, df2 = 24, p-value = 3.917e-05


In [31]:
# modelo

###Modelo de regresión múltiple
mod1=lm(PIBpc~GCF+GCFG+FBK, x=T)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

#Pruebas para detectar autocorrelacionados. 
# Test de Breush Godfrey
bgtest(mod1, order = 2, order.by = NULL, type = c("Chisq"),
data = datos)

# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)

################
# Para corregir autocorreción 
# a) modelo en diferencia, b) agregar variable de tendencia

# a) Variables en diferencia

dPIBpc = diff(PIBpc)
dPIBpc

dGCF = diff(GCF)
dGCF

dGCFG = diff(GCFG)
dGCFG

dFBK = diff(FBK)
dFBK

###Modelo de regresión múltiple
mod1=lm(dPIBpc~dGCF+dGCFG+dFBK, x=T)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

#Pruebas para detectar autocorrelacionados. 
# Test de Breush Godfrey
bgtest(mod1, order = 2, order.by = NULL, type = c("Chisq"),
data = datos)

# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)

# b) Agregar variable de tendencia

# modelo

###Modelo de regresión múltiple
mod1=lm(PIBpc~GCF+GCFG+FBK+year, x=T)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

#Pruebas para detectar autocorrelacionados. 
# Test de Breush Godfrey
bgtest(mod1, order = 2, order.by = NULL, type = c("Chisq"),
data = datos)

# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)

################
# Para corregir heterocedasticidad
# a) MCG, b) Aplicar transformación logarítmica

# a) MCG


###Modelo de regresión múltiple


mod1= gls(PIBpc~GCF+GCFG+FBK)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

# b) Aplicar transformación logarítmica

#Variables en logaritmo

lPIBpc = log(PIBpc)
lPIBpc

lGCF = log(GCF)
lGCF

lGCFG = log(GCFG)
lGCFG

lFBK = log(FBK)
lFBK

###Modelo de regresión múltiple
mod1=lm(lPIBpc~lGCF+lGCFG+lFBK, x=T)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

#Pruebas para detectar heterocedásticidad 
# Prueba de Breush-Pagan-Godfrey (BPG)
bptest(mod1)

# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)


Call:
lm(formula = PIBpc ~ GCF + GCFG + FBK, x = T)

Residuals:
       Min         1Q     Median         3Q        Max 
-7.974e+09 -3.041e+09  2.045e+08  2.650e+09  7.312e+09 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.661e+10  1.201e+10   1.383  0.17837    
GCF         1.007e+00  2.027e-01   4.968 3.66e-05 ***
GCFG        1.451e+00  4.034e-01   3.596  0.00133 ** 
FBK         3.216e-01  2.011e-01   1.600  0.12176    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.284e+09 on 26 degrees of freedom
Multiple R-squared:  0.9973,	Adjusted R-squared:  0.997 
F-statistic:  3234 on 3 and 26 DF,  p-value: < 2.2e-16



	Shapiro-Wilk normality test

data:  res
W = 0.96895, p-value = 0.511


GCF      GCFG       FBK 
157.25474  54.23371  42.03326


	Breusch-Godfrey test for serial correlation of order up to 2

data:  mod1
LM test = 24.86, df = 2, p-value = 3.996e-06



	RESET test

data:  mod1
RESET = 15.953, df1 = 2, df2 = 24, p-value = 3.917e-05


...


Call:
lm(formula = lPIBpc ~ lGCF + lGCFG + lFBK, x = T)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.033733 -0.009588  0.001040  0.009977  0.029287 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.418192   0.735000   0.569    0.574    
lGCF        0.880241   0.061528  14.306 7.76e-14 ***
lGCFG       0.120455   0.023475   5.131 2.38e-05 ***
lFBK        0.006593   0.021210   0.311    0.758    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01743 on 26 degrees of freedom
Multiple R-squared:  0.9971,	Adjusted R-squared:  0.9968 
F-statistic:  3027 on 3 and 26 DF,  p-value: < 2.2e-16



	Shapiro-Wilk normality test

data:  res
W = 0.97824, p-value = 0.7772


lGCF    lGCFG     lFBK 
29.26055 11.77049 11.71404


	studentized Breusch-Pagan test

data:  mod1
BP = 5.6917, df = 3, p-value = 0.1276



	RESET test

data:  mod1
RESET = 11.929, df1 = 2, df2 = 24, p-value = 0.000253
